In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip3 install ipdb

Mounted at /content/drive
     |████████████████████████████████| 791 kB 4.3 MB/s 
     |████████████████████████████████| 374 kB 56.1 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=437008717480d9498760d0fc7b53f9fb304ec2b2918f7c19b9649f1cf4bd1cf4
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, bu

In [ ]:
!mkdir rob535data

!cp drive/Shareddrives/ROB\ 535\ -\ Perception/*data.zip /content/rob535data/

!unzip ./rob535data/*.zip

In [2]:
%cd drive/Shareddrives/ROB\ 535\ -\ Perception
!pwd

/content/drive/Shareddrives/ROB 535 - Perception
/content/drive/Shareddrives/ROB 535 - Perception


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from model import Yolov1
from dataset import GTADataset
from utils.utils import (
    mean_average_precision,
    get_bboxes,
    load_checkpoint,
)
from loss import YoloLoss
import time
from glob import glob
from PIL import Image

LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0

LOAD_MODEL_FILE = './checkpoints/overfit130.pth.tar'

model = Yolov1(split_size=7, num_boxes=2, num_classes=3).to(DEVICE)
optimizer = optim.Adam(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)
loss_fn = YoloLoss(S=7, B=2, C=3)

load_checkpoint(torch.load(LOAD_MODEL_FILE, map_location=torch.device(DEVICE)), model, optimizer)

=> Loading checkpoint


In [4]:
import os
import csv

In [9]:
test_files = glob('/content/test/*/*_image.jpg')
test_files.sort()
model.eval()
i = 0
name = 'results_labels.csv'
threshold = 0.0
with open(name, 'w') as f:
    writer = csv.writer(f, delimiter=',', lineterminator='\n')
    writer.writerow(['guid/image', 'label'])
    for test_file in test_files:
        guid = test_file.split('/')[-2]
        idx = test_file.split('/')[-1].replace('_image.jpg', '')
        image = Image.open(test_file)
        x = torchvision.transforms.ToTensor()(image).to(DEVICE)
        
        x = x.unsqueeze(0)
        # import ipdb
        # ipdb.set_trace()
        out = model(x)
        i = i + 1
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"Image Processing:{i}/{len(test_files)}")
        

        out = out.reshape((7,7,13))
        prob1 = out[:,:,3]
        prob2 = out[:,:,8]
        prob = torch.max(prob1, prob2).max()
        if prob < threshold:
            res = 0
        else:
            #res = torch.argmax(out[:, :, [0,1,2]].reshape(-1)).item() % 3
            idx1 = torch.max(prob1, prob2).argmax() // 7
            idx2 = torch.max(prob1, prob2).argmax() - idx1 * 7
            #ipdb.set_trace()

            res = out[idx1, idx2, 0:3].argmax().item()

        #ipdb.set_trace()
        
        #ipdb.set_trace()
        #res = out[..., :3].argmax(-1).unsqueeze(-1)

        print(f"index will be {idx1} {idx2}", )
        print(f"result is {res}")
        writer.writerow(['{}/{}'.format(guid, idx), res])
    #ipdb.set_trace()

Image Processing:1/2631
index will be 3 3
result is 0
Image Processing:2/2631
index will be 3 3
result is 0
Image Processing:3/2631
index will be 3 3
result is 0
Image Processing:4/2631
index will be 3 3
result is 0
Image Processing:5/2631
index will be 3 3
result is 0
Image Processing:6/2631
index will be 3 3
result is 0
Image Processing:7/2631
index will be 3 3
result is 0
Image Processing:8/2631
index will be 3 3
result is 0
Image Processing:9/2631
index will be 3 3
result is 0
Image Processing:10/2631
index will be 3 3
result is 0
Image Processing:11/2631
index will be 3 3
result is 0
Image Processing:12/2631
index will be 3 3
result is 0
Image Processing:13/2631
index will be 3 3
result is 0
Image Processing:14/2631
index will be 3 3
result is 0
Image Processing:15/2631
index will be 3 3
result is 0
Image Processing:16/2631
index will be 3 3
result is 0
Image Processing:17/2631
index will be 3 3
result is 0
Image Processing:18/2631
index will be 3 3
result is 0
Image Processing:19

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


Streaming output truncated to the last 5000 lines.
index will be 3 3
result is 1
Image Processing:966/2631
index will be 3 3
result is 1
Image Processing:967/2631
index will be 3 3
result is 1
Image Processing:968/2631
index will be 3 3
result is 1
Image Processing:969/2631
index will be 3 3
result is 1
Image Processing:970/2631
index will be 3 3
result is 1
Image Processing:971/2631
index will be 3 3
result is 1
Image Processing:972/2631
index will be 3 3
result is 1
Image Processing:973/2631
index will be 3 3
result is 1
Image Processing:974/2631
index will be 1 3
result is 1
Image Processing:975/2631
index will be 1 3
result is 0
Image Processing:976/2631
index will be 1 3
result is 0
Image Processing:977/2631
index will be 1 3
result is 0
Image Processing:978/2631
index will be 1 3
result is 0
Image Processing:979/2631
index will be 1 3
result is 0
Image Processing:980/2631
index will be 1 3
result is 0
Image Processing:981/2631
index will be 1 3
result is 0
Image Processing:982/26

In [ ]:
x.shape

torch.Size([3, 448, 448])